## Reconhecimento de Espécies Florestais por Visão Computacional em Ambientes com Recursos Computacionais Limitados


### Desenvolvido por  

**André Rizzo**  
Cientista de Dados Sênior | Estatístico | MBA em IA e Big Data (USP)  
andrerizzo@hotmail.com  
http://www.linkedin.com/in/andrerizzo1  

### 1. Preparação do Ambiente

#### 1.1. Importação das Bibliotecas Necessárias

In [18]:
from src.data_acquisition import *
from src.preprocess import *
from src.build_model import *
from src.train_model import *
from src.evaluate_model import *

#### 1.2. Definição das Variáveis

In [15]:
# GOOGLE COLAB
#UPLOAD_PATH = '/content'
#PATH_DESTINO = '/images'
#FILES_TO_DELETE_PATH = '/content/patches/class_9/0939*'
# Localização dos patches
#IMAGES_PATH = 'patches'
#Localização dos patches já divididos em treino, validação e teste
#train_path = r'C:\Users\andre\teste\train'
#val_path = r'C:\Users\andre\teste\val'
#test_path = r'C:\Users\andre\teste\test'


# MÁQUINA LOCAL
# Local onde os patches devem ser armazenados
PATH_DESTINO = r'C:\Users\andre\teste'
# Localização dos patches a serem apagados
FILES_TO_DELETE_PATH = r'C:\Users\andre\teste\patches\class_9\0939*'
# Localização dos patches
IMAGES_PATH = r'C:\Users\andre\teste\patches'
#Localização dos patches já divididos em treino, validação e teste
train_path = r'C:\Users\andre\teste\train'
val_path = r'C:\Users\andre\teste\val'
test_path = r'C:\Users\andre\teste\test'




---

<br>

### 2. Carregar Imagens

#### 2.1. Baixar os patches das imagens 

In [4]:
ID_GOOGLE_IMAGES = ['1RajkAeTumEVBAbrq3BIVj8ELFm2USifw',
                    '1Sg0ZuRN5vRASb13mp5JVSkvRLeko_Mp8',
                    '1iPHhbOb7X8dc9c0PLj1uaw5PEmAUVrGh',
                    '1SHCsgrARnZIEZ43gauQf-8ycsk9OV3xy',
                    '18PW2ac7hGe494TAkGzg3IGEntktT6f9Q',
                    '1M9qKLrQ-Z98FTTW9Jo2xcpwjwSfHj3WN',
                    '1vGa8kdjsTlYQgM7mJzpteoKNE5IsGsz5']
n = 1
for id in ID_GOOGLE_IMAGES:
    zip_file = download_patches_file(id_google_drive_file=id,
                          path_destino=PATH_DESTINO,
                          arquivo_destino=f'imagens{n}.zip'
                          )
    uncompress_patches(arquivo_zip=zip_file, path_destino=PATH_DESTINO)
    n=n+1




 Arquivos descompactados com sucesso !


In [8]:
files_to_delete = glob.glob(FILES_TO_DELETE_PATH)

for file in files_to_delete:
    os.remove(file)
print(f'Arquivos deletados com sucesso !')

Arquivos deletados com sucesso !


#### 2.2. Gera lista das imagens existentes e respectivas classes

In [7]:
X_temp = gera_lista(image_path=IMAGES_PATH)

X = X_temp['imagem']
y = X_temp['classe']

#### 2.3. Gera lista de imagens que serão usadas para **treino**, **teste** e **validação**

In [8]:
X_train, X_test = train_test_val(image_ds=X, tam_teste=0.1,
                                 estratifica=y)
X_train, X_val = train_test_val(image_ds=X_train, tam_teste=0.2,
                                estratifica=y[X_train.index])

print(f'Número de imagens para treinamento: {len(X_train)}')
print(f'Número de imagens para validação: {len(X_val)}')
print(f'Número de imagens para teste: {len(X_test)}')

Número de imagens para treinamento: 2116
Número de imagens para validação: 530
Número de imagens para teste: 295


#### 2.4. Criação dos diretórios onde os symlinks serão armazenados

In [9]:
train_path, test_path, val_path =  folder_creation(root_patches_folder=PATH_DESTINO)

Diretórios de treinamento criados com sucesso ! 

Diretórios de validação criados com sucesso ! 

Diretórios de teste criados com sucesso ! 



#### 2.5. Criação dos symlinks

In [10]:
# Cria symlinks para imagens de treinamento
origem = IMAGES_PATH
destino = train_path
symlink(X_train, origem, destino)


# Cria symlinks para imagens de validação
origem =IMAGES_PATH
destino = val_path
symlink(X_val, origem, destino)


# Cria symlinks para imagens de teste
origem = IMAGES_PATH
destino = test_path
symlink(X_test, origem, destino)

Creating symlinks: 100%|██████████| 295/295 [00:28<00:00, 10.21it/s]


---

<br>

### 3. Pré-Processamento dos Dados

#### 3.1. Criação dos datasets de treino, validação e teste

In [10]:
train_ds = gera_dataset_train_test_val(train_path,
                                       classes='inferred',
                                       tipo_classes='categorical',
                                       tam_batch=32,
                                       tam_imagem=(224,224),
                                       aleatorio=True,
                                       percent_val=None,
                                       subconjunto=None,
                                       symlink=True
                                       )

val_ds = gera_dataset_train_test_val(val_path,
                                     classes='inferred',
                                     tipo_classes='categorical',
                                     tam_batch=32,
                                     tam_imagem=(224,224),
                                     aleatorio=False,
                                     percent_val=None,
                                     subconjunto=None,
                                     symlink=True
                                    )

test_ds = gera_dataset_train_test_val(test_path,
                                      classes='inferred',
                                      tipo_classes='categorical',
                                      tam_batch=32,
                                      tam_imagem=(224,224),
                                      aleatorio=False,
                                      percent_val=None,
                                      subconjunto=None,
                                      symlink=True
                                       )

Found 296240 files belonging to 41 classes.
Found 74200 files belonging to 41 classes.
Found 41300 files belonging to 41 classes.


In [11]:
train_class_names = train_ds.class_names
print("Train classes:", train_class_names)

Train classes: ['class_01', 'class_02', 'class_03', 'class_04', 'class_05', 'class_06', 'class_07', 'class_08', 'class_09', 'class_10', 'class_11', 'class_12', 'class_13', 'class_14', 'class_15', 'class_16', 'class_17', 'class_18', 'class_19', 'class_20', 'class_21', 'class_22', 'class_23', 'class_24', 'class_25', 'class_26', 'class_27', 'class_28', 'class_29', 'class_30', 'class_31', 'class_32', 'class_33', 'class_34', 'class_35', 'class_36', 'class_37', 'class_38', 'class_39', 'class_40', 'class_41']


#### 3.2. Normalização dos datasets

In [12]:
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input as mobilenetv3_preprocess

train_images_norm = preprocess(dataset=train_ds, 
                               preprocessing_fn=mobilenetv3_preprocess)

val_images_norm = preprocess(dataset=val_ds,
                             preprocessing_fn=mobilenetv3_preprocess)

test_images_norm = preprocess(dataset=test_ds,
                              preprocessing_fn=mobilenetv3_preprocess)

---

<br>

### 4. Modelo MobileNet v3 Large v1

#### 4.1. Criação e compilação do modelo MobileNetv3 Large v1

In [ ]:
model_mobilenetv3_large_v1 = build_model_mobilenetv3_large_v1(formato_imagem=(224,224,3), num_classes=41)
model_mobilenetv3_large_v1 = compile_model_mobilenetv3(modelo=model_mobilenetv3_large_v1, lr_rate=0.0001)
model_mobilenetv3_large_v1.summary()

#### 4.2. Treinamento do modelo MobileNetv3 Large v1

In [ ]:
history_mobilenet_large_v1 = train_model(model=model_mobilenetv3_large_v1,
                      train_images=train_images_norm,
                      val_images=val_images_norm,
                      output_dir='models',
                      model_file_name='MobileNetv3_large_v1_.keras',
                      epochs=20,
                      patience=5,
                      min_lr_ROP=0.00001,
                      factorROP=0.5,
                      patienceROP=3
                      )

12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv (Conv2D)       │ (None, 112, 112,  │        432 │ rescaling[0][0]   │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_bn             │ (None, 112, 112,  │         64 │ conv[0][0]        │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 112, 112,  │          0 │ conv_bn[0][0]     │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        144 │ activation[0][0]  │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │         64 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        256 │ re_lu[0][0]       │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_add   │ (None, 112, 112,  │          0 │ activation[0][0], │
│ (Add)               │ 16)               │            │ expanded_conv_pr… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, 112, 112,  │      1,024 │ expanded_conv_ad… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, 112, 112,  │        256 │ expanded_conv_1_… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 112, 112,  │          0 │ expanded_conv_1_… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 113, 113,  │          0 │ re_lu_1[0][0]     │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 56, 56,    │        576 │ expanded_conv_1_… │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 56, 56,    │        256 │ expanded_conv_1_

 Total params: 3,630,249 (13.85 MB)

 Trainable params: 633,897 (2.42 MB)

 Non-trainable params: 2,996,352 (11.43 MB)

#### 4.3. Avaliação do modelo MobileNetv3 Large v1

In [ ]:
plot_training_history(history=history_mobilenet_large_v1)
performance_metrics(model=model_mobilenetv3_large_v1,
                    test_dataset=test_images_norm,
                    class_names=train_class_names
                    )

---

<br>

### 5. Modelo MobileNet v3 Large v2

#### 5.1. Criação e compilação do modelo

In [19]:
model_mobilenetv3_large_v2 = build_model_mobilenetv3_large_v2(formato_imagem=(224,224,3), num_classes=41)
model_mobilenetv3_large_v2 = compile_model_mobilenetv3(modelo=model_mobilenetv3_large_v2, lr_rate=0.0001)
model_mobilenetv3_large_v2.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv (Conv2D)       │ (None, 112, 112,  │        432 │ rescaling_1[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_bn             │ (None, 112, 112,  │         64 │ conv[0][0]        │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_20       │ (None, 112, 112,  │          0 │ conv_bn[0][0]     │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        144 │ activation_20[0]… │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │         64 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_19 (ReLU)     │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        256 │ re_lu_19[0][0]    │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_add   │ (None, 112, 112,  │          0 │ activation_20[0]… │
│ (Add)               │ 16)               │            │ expanded_conv_pr… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, 112, 112,  │      1,024 │ expanded_conv_ad… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, 112, 112,  │        256 │ expanded_conv_1_… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_20 (ReLU)     │ (None, 112, 112,  │          0 │ expanded_conv_1_… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 113, 113,  │          0 │ re_lu_20[0][0]    │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 56, 56,    │        576 │ expanded_conv_1_… │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 56, 56,    │        256 │ expanded_conv_1_

 Total params: 3,630,249 (13.85 MB)

 Trainable params: 3,605,849 (13.76 MB)

 Non-trainable params: 24,400 (95.31 KB)

#### 5.2. Treinamento do modelo MobileNetv3 Large v2

In [ ]:
history_mobilenet_large_v2 = train_model(model=model_mobilenetv3_large_v2,
                      train_images=train_images_norm,
                      val_images=val_images_norm,
                      output_dir='models',
                      model_file_name='MobileNetv3_large_v2_.keras',
                      epochs=20,
                      patience=5,
                      min_lr_ROP=0.00001,
                      factorROP=0.5,
                      patienceROP=3
                      )
                      

#### 5.3. Avaliação do modelo MobileNetv3 Large v2

In [ ]:
plot_training_history(history=history_mobilenet_large_v2)
performance_metrics(model=model_mobilenetv3_large_v2,
                    test_dataset=test_images_norm,
                    class_names=train_class_names
                    )

---

<br>

### 6. Modelo MobileNetv3 Small v1

#### 6.1. Criação e compilação do modelo

In [ ]:
model_mobilenetv3_small_v1 = build_model_mobilenetv3_small_v1(formato_imagem=(224,224,3), num_classes=41)
model_mobilenetv3_small_v1 = compile_model_mobilenetv3(modelo=model_mobilenetv3_small_v1, lr_rate=0.0001)
model_mobilenetv3_small_v1.summary()

#### 6.2. Treinamento do modelo MobileNetv3 Large v2

In [ ]:
history_mobilenet_small_v1 = train_model(model=model_mobilenetv3_small_v1,
                      train_images=train_images_norm,
                      val_images=val_images_norm,
                      output_dir='models',
                      model_file_name='MobileNetv3_small_v1_.keras',
                      epochs=20,
                      patience=5,
                      min_lr_ROP=0.00001,
                      factorROP=0.5,
                      patienceROP=3
                      )
                      

#### 6.3. Avaliação do modelo MobileNetv3 Small v1

In [ ]:
plot_training_history(history=history_mobilenet_small_v1)
performance_metrics(model=model_mobilenetv3_small_v1,
                    test_dataset=test_images_norm,
                    class_names=train_class_names
                    )